## CORTEVA CODING EXERCISE (Problem 1,2,3)
  - Main direction for me to create a table and ingest data to the database was to make the automatic process with several functions and call the function where it requires. So that reduce time.


In [1]:
# Libraries
import pandas as pd
import numpy as np
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import os
import re
import getpass

mypasswd = getpass.getpass()
username = 'mk7zp'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

········


In [2]:
# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=True)
del mypasswd

In [3]:
# This function is to print all the existing tables in the given schema.
def list_all_tables(schemaname):
    query ='''
        SELECT *
        FROM pg_catalog.pg_tables
        WHERE schemaname = '{schemaname}';
    '''.format(schemaname = schemaname)
    with engine.connect() as connection:
        res = connection.execute(query)
    for result in res:
        print(result)

In [4]:
def drop_table(table_name):
    query = 'DROP TABLE IF EXISTS "{table_name}";'
    with engine.connect() as connection:
        res = connection.execute(query.format(table_name=table_name))

In [5]:
def data_type(table_name):
    query ='''
    SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS 
    where table_schema = 'mk7zp' and table_name = '{table_name}';
    '''.format(table_name = table_name)
    with engine.connect() as connection:
         d_type = pd.read_sql(query, connection)
    return d_type

In [6]:
def create_table_stmt(new_table_name, dataframe, key1, key2=''):
    createTableStmt = "CREATE TABLE " + new_table_name + "(\n"
    dtype2SQL = {'object' : 'TEXT', 'float64' : 'REAL', 'int64' : 'BIGINT', 'datetime64[ns]':'TIMESTAMP'}
    columnList = list(dataframe) 
    for columnName in columnList:
        pandaType = str(dataframe[columnName].dtype) # we need to force the conversion of the type name to a string.
        sqlDataTypeStr = dtype2SQL[pandaType] 
        createTableStmt += "\t{} {},\n".format(columnName, sqlDataTypeStr)
    if key2 == '':
        createTableStmt += "\tPRIMARY KEY({})\n".format(key1)
    else:
        createTableStmt += "\tPRIMARY KEY({},{})\n".format(key1, key2)
    createTableStmt += ");"  

    print(len(createTableStmt))  

    with engine.connect() as connection:
        connection.execute(createTableStmt)


In [7]:
def ingest_data(new_table_name, dataframe):
    print(dataframe.columns)
    print('\t')
    colstring = ','.join(dataframe).replace(' ', '_')   # variables of the table
    print(colstring)
    str_specifier = ','.join(['%s'] * len(dataframe.columns))  # the values places of variables
    print(str_specifier)
    
    INSERT_SQL = f'INSERT INTO {new_table_name}'
    INSERT_SQL += f' ({colstring}) values ({str_specifier})'

    print(INSERT_SQL)

    # Note: The Commit Will Be Automatic after this with clause
    with engine.connect() as connection:
        for row in dataframe.itertuples(index=False, name=None):  # pull each row as a tuple     
    #         print(row) 
            connection.execute(INSERT_SQL,row) # insert row


In [8]:
def display_db_table(table_name):
    query = ''' SELECT * FROM {table_name}; '''.format(table_name = table_name)
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

### Problem 1, 2  - Design Model and Ingest Data

### Corn Yield dataset

In [9]:
yld_file = './yld_data/US_corn_grain_yield.txt'
df_yld = pd.read_csv(yld_file, sep = '\t', names = ['year', 'hvst_corn_mgt'])  # in 1000s of megatons 

In [10]:
df_yld.head(2)

,year,hvst_corn_mgt
0,1985,225447
1,1986,208944


In [16]:
# list_all_tables('mk7zp')
# drop_table('corn_yield')


In [12]:
df_yld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   year           30 non-null     int64
 1   hvst_corn_mgt  30 non-null     int64
dtypes: int64(2)
memory usage: 608.0 bytes


#### Design a Table

In [13]:
# Data Model 1
# statement = """
# CREATE TABLE IF NOT EXISTS corn_yield (
#  year BIGINT PRIMARY KEY,
#  hvst_corn_mgt BIGINT,
#  )
# """

# Data Model 2 with a pre-defined function
create_table_stmt('corn_yield', df_yld, 'year')

83
2022-07-14 18:48:25,455 INFO sqlalchemy.engine.base.Engine CREATE TABLE corn_yield(
	year BIGINT,
	hvst_corn_mgt BIGINT,
	PRIMARY KEY(year)
);
2022-07-14 18:48:25,456 INFO sqlalchemy.engine.base.Engine {}
2022-07-14 18:48:25,472 INFO sqlalchemy.engine.base.Engine COMMIT


#### Ingest Data 

In [14]:
ingest_data('corn_yield', df_yld)

Index(['year', 'hvst_corn_mgt'], dtype='object')
	
year,hvst_corn_mgt
%s,%s
INSERT INTO corn_yield (year,hvst_corn_mgt) values (%s,%s)
2022-07-14 16:43:18,681 INFO sqlalchemy.engine.base.Engine INSERT INTO corn_yield (year,hvst_corn_mgt) values (%s,%s)
2022-07-14 16:43:18,682 INFO sqlalchemy.engine.base.Engine (1985, 225447)
2022-07-14 16:43:18,685 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 16:43:18,689 INFO sqlalchemy.engine.base.Engine INSERT INTO corn_yield (year,hvst_corn_mgt) values (%s,%s)
2022-07-14 16:43:18,690 INFO sqlalchemy.engine.base.Engine (1986, 208944)
2022-07-14 16:43:18,692 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 16:43:18,694 INFO sqlalchemy.engine.base.Engine INSERT INTO corn_yield (year,hvst_corn_mgt) values (%s,%s)
2022-07-14 16:43:18,695 INFO sqlalchemy.engine.base.Engine (1987, 181143)
2022-07-14 16:43:18,696 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 16:43:18,698 INFO sqlalchemy.engine.base.Engine INSERT INTO corn_yield (year,hvs

In [15]:
df = display_db_table('corn_yield')

2022-07-14 16:43:20,883 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-14 16:43:20,884 INFO sqlalchemy.engine.base.Engine {'name': ' SELECT * FROM corn_yield; '}
2022-07-14 16:43:20,887 INFO sqlalchemy.engine.base.Engine  SELECT * FROM corn_yield; 
2022-07-14 16:43:20,888 INFO sqlalchemy.engine.base.Engine {}


In [16]:
df.head()

,year,hvst_corn_mgt
0,1985,225447
1,1986,208944
2,1987,181143
3,1988,125194
4,1989,191320


In [17]:
df.shape

(30, 2)

### Weather - Design Model and Ingest Data

In [18]:
wx_dir = './wx_data/'
wx_list = [file for file in os.listdir(wx_dir)]

In [19]:
len(wx_list)  # total number of weather file; the number of weather station 

167

#### Method 1: create a table and insert into values

In [20]:
# drop_table('weather')

In [21]:
# to get column name and data type to create a table
df_wx_samp = pd.read_csv(wx_dir + wx_list[1], sep = "\t", names = ['date','max_temp_c','min_temp_c','precip_mm'])
df_wx_samp['date'] = pd.to_datetime(df_wx_samp['date'], format='%Y%m%d')
df_wx_samp['station'] = wx_list[1][:-4]
df_wx_samp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10957 entries, 0 to 10956
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        10957 non-null  datetime64[ns]
 1   max_temp_c  10957 non-null  int64         
 2   min_temp_c  10957 non-null  int64         
 3   precip_mm   10957 non-null  int64         
 4   station     10957 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 428.1+ KB


In [22]:
# create a model using predefined function
create_table_stmt('weather',df_wx_samp, 'date','station') # db table name, dataframe, primary key1, key2

142
2022-07-14 16:43:50,998 INFO sqlalchemy.engine.base.Engine CREATE TABLE weather(
	date TIMESTAMP,
	max_temp_c BIGINT,
	min_temp_c BIGINT,
	precip_mm BIGINT,
	station TEXT,
	PRIMARY KEY(date,station)
);
2022-07-14 16:43:50,999 INFO sqlalchemy.engine.base.Engine {}
2022-07-14 16:43:51,035 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
# ingest data using 'INSERT INTO VALUE' method

# start time: 2022-07-14 13:58:34,866 
# ending time: it is over 30 mins. took too much time.Stoped.

# total_rows = list()       # to check total row after combinig all tables and compare with db table after injesting
# for file in wx_list: 
#     df_wx = pd.read_csv(wx_dir + file, sep = "\t", names = ['date','max_temp_c','min_temp_c','precip_mm'])
#     df_wx['date'] = pd.to_datetime(df_wx['date'], format='%Y%m%d')
#     df_wx['station'] = file[:-4]
#     total_rows.append(len(df_wx))
#     ingest_data('weather',df_wx)

  

####  Method 2: using to_sql

In [ ]:
total_rows = list()       # to check total row after combinig all tables and compare with db table after injesting
for file in wx_list: 
    df_wx = pd.read_csv(wx_dir + file, sep = "\t", names = ['date','max_temp_c','min_temp_c','precip_mm'])
    df_wx = df_wx.replace({-9999:None})
    df_wx['date'] = pd.to_datetime(df_wx['date'], format='%Y%m%d')
    df_wx['station'] = file[:-4]
    total_rows.append(len(df_wx))
    df_wx.to_sql('weather', engine, schema= username, if_exists='append', index=False, chunksize=50) 

# start time: 2022-07-14 14:50:55
# ending time: 2022-07-14 14:57:26,480 INFO sqlalchemy.engine.base.Engine COMMIT


2022-07-14 16:43:56,007 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-07-14 16:43:56,008 INFO sqlalchemy.engine.base.Engine {'schema': 'mk7zp', 'name': 'weather'}
2022-07-14 16:43:56,020 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2022-07-14 16:43:56,022 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:56,022 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(1985, 1, 1, 0, 0), 'max_temp_c': 150, 'min_temp_c': -6, 'precip_mm': 20, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 1, 2, 0, 0), 'max_temp_c': -6, 'min_temp_c': -33, 'precip_mm': 38, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 1, 3, 0, 0), 'max_temp_c': -11, 'min_temp_c': -50, 'precip_mm': 0, 'station': 'USC0

2022-07-14 16:43:56,159 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:56,160 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(1985, 9, 8, 0, 0), 'max_temp_c': 300, 'min_temp_c': 211, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 9, 9, 0, 0), 'max_temp_c': 278, 'min_temp_c': 194, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 9, 10, 0, 0), 'max_temp_c': 239, 'min_temp_c': 172, 'precip_mm': 30, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 9, 11, 0, 0), 'max_temp_c': 189, 'min_temp_c': 78, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 9, 12, 0, 0), 'max_temp_c': 161, 'min_temp_c': 67, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1985, 9, 13, 0, 0), 'max_temp_c': 161, 'min_temp_c': 39, 'prec

2022-07-14 16:43:56,212 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(1986, 5, 16, 0, 0), 'max_temp_c': 256, 'min_temp_c': 178, 'precip_mm': 81, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 17, 0, 0), 'max_temp_c': 261, 'min_temp_c': 139, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 18, 0, 0), 'max_temp_c': 278, 'min_temp_c': 189, 'precip_mm': 81, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 19, 0, 0), 'max_temp_c': 200, 'min_temp_c': 128, 'precip_mm': 99, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 20, 0, 0), 'max_temp_c': 128, 'min_temp_c': 83, 'precip_mm': 10, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 21, 0, 0), 'max_temp_c': 122, 'min_temp_c': 78, 'precip_mm': 8, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 22, 0, 0), 'max_temp_c': 122, 'min_temp_c': 83, 'precip_mm': 15, 'station': 'USC00339312'}, {'date': datetime.datetime(1986, 5, 23, 0,

2022-07-14 16:43:56,273 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:56,274 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(1987, 3, 12, 0, 0), 'max_temp_c': 50, 'min_temp_c': -39, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 3, 13, 0, 0), 'max_temp_c': 50, 'min_temp_c': -61, 'precip_mm': 28, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 3, 14, 0, 0), 'max_temp_c': 33, 'min_temp_c': -28, 'precip_mm': 38, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 3, 15, 0, 0), 'max_temp_c': 39, 'min_temp_c': -17, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 3, 16, 0, 0), 'max_temp_c': 44, 'min_temp_c': -50, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 3, 17, 0, 0), 'max_temp_c': 78, 'min_temp_c': -72, 'prec

2022-07-14 16:43:56,324 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(1987, 11, 17, 0, 0), 'max_temp_c': 167, 'min_temp_c': 117, 'precip_mm': 30, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 18, 0, 0), 'max_temp_c': 117, 'min_temp_c': 33, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 19, 0, 0), 'max_temp_c': 78, 'min_temp_c': -11, 'precip_mm': 5, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 20, 0, 0), 'max_temp_c': 44, 'min_temp_c': -56, 'precip_mm': 8, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 21, 0, 0), 'max_temp_c': -39, 'min_temp_c': -89, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 22, 0, 0), 'max_temp_c': 50, 'min_temp_c': -106, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 23, 0, 0), 'max_temp_c': 111, 'min_temp_c': 11, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(1987, 11, 24,

...

2022-07-14 16:43:57,768 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:57,768 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2006, 10, 8, 0, 0), 'max_temp_c': 183, 'min_temp_c': 11, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2006, 10, 9, 0, 0), 'max_temp_c': 239, 'min_temp_c': 50, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2006, 10, 10, 0, 0), 'max_temp_c': 244, 'min_temp_c': 83, 'precip_mm': 15, 'station': 'USC00339312'}, {'date': datetime.datetime(2006, 10, 11, 0, 0), 'max_temp_c': 222, 'min_temp_c': 100, 'precip_mm': 3, 'station': 'USC00339312'}, {'date': datetime.datetime(2006, 10, 12, 0, 0), 'max_temp_c': 189, 'min_temp_c': 139, 'precip_mm': 109, 'station': 'USC00339312'}, {'date': datetime.datetime(2006, 10, 13, 0, 0), 'max_temp_c': 139, 'min_temp_c': -6

2022-07-14 16:43:57,826 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2007, 6, 15, 0, 0), 'max_temp_c': 272, 'min_temp_c': 139, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 16, 0, 0), 'max_temp_c': 261, 'min_temp_c': 122, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 17, 0, 0), 'max_temp_c': 278, 'min_temp_c': 128, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 18, 0, 0), 'max_temp_c': 322, 'min_temp_c': 156, 'precip_mm': 30, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 19, 0, 0), 'max_temp_c': 317, 'min_temp_c': 172, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 20, 0, 0), 'max_temp_c': 267, 'min_temp_c': 183, 'precip_mm': 61, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 21, 0, 0), 'max_temp_c': 256, 'min_temp_c': 111, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2007, 6, 22, 0,

2022-07-14 16:43:57,892 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:57,892 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2008, 4, 10, 0, 0), 'max_temp_c': 200, 'min_temp_c': 67, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 4, 11, 0, 0), 'max_temp_c': 178, 'min_temp_c': 11, 'precip_mm': 61, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 4, 12, 0, 0), 'max_temp_c': 222, 'min_temp_c': 117, 'precip_mm': 28, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 4, 13, 0, 0), 'max_temp_c': 117, 'min_temp_c': 39, 'precip_mm': 3, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 4, 14, 0, 0), 'max_temp_c': 67, 'min_temp_c': 17, 'precip_mm': 20, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 4, 15, 0, 0), 'max_temp_c': 89, 'min_temp_c': 0, 'preci

2022-07-14 16:43:57,955 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2008, 12, 16, 0, 0), 'max_temp_c': 106, 'min_temp_c': -78, 'precip_mm': 23, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 17, 0, 0), 'max_temp_c': -6, 'min_temp_c': -78, 'precip_mm': 5, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 18, 0, 0), 'max_temp_c': -11, 'min_temp_c': -44, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 19, 0, 0), 'max_temp_c': 0, 'min_temp_c': -39, 'precip_mm': 122, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 20, 0, 0), 'max_temp_c': 78, 'min_temp_c': -44, 'precip_mm': 147, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 21, 0, 0), 'max_temp_c': -11, 'min_temp_c': -44, 'precip_mm': 10, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 12, 22, 0, 0), 'max_temp_c': -39, 'min_temp_c': -183, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2008, 1

2022-07-14 16:43:58,014 INFO sqlalchemy.engine.base.Engine INSERT INTO mk7zp.weather (date, max_temp_c, min_temp_c, precip_mm, station) VALUES (%(date)s, %(max_temp_c)s, %(min_temp_c)s, %(precip_mm)s, %(station)s)
2022-07-14 16:43:58,014 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2009, 10, 12, 0, 0), 'max_temp_c': 128, 'min_temp_c': 0, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2009, 10, 13, 0, 0), 'max_temp_c': 128, 'min_temp_c': 22, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2009, 10, 14, 0, 0), 'max_temp_c': 122, 'min_temp_c': -6, 'precip_mm': 0, 'station': 'USC00339312'}, {'date': datetime.datetime(2009, 10, 15, 0, 0), 'max_temp_c': 61, 'min_temp_c': 33, 'precip_mm': 43, 'station': 'USC00339312'}, {'date': datetime.datetime(2009, 10, 16, 0, 0), 'max_temp_c': 61, 'min_temp_c': 17, 'precip_mm': 64, 'station': 'USC00339312'}, {'date': datetime.datetime(2009, 10, 17, 0, 0), 'max_temp_c': 67, 'min_temp_c': 17, 'pr

In [27]:
print("total rows: ", sum(total_rows)) # total row of combined weather dataframe
# total rows:  1729957

total rows:  1729957


In [28]:
db_wx = display_db_table('weather')
db_wx.shape # (1729957, 5) Same total rows from pandas datafram.

2022-07-14 15:00:19,875 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-14 15:00:19,876 INFO sqlalchemy.engine.base.Engine {'name': ' SELECT * FROM weather; '}
2022-07-14 15:00:19,879 INFO sqlalchemy.engine.base.Engine  SELECT * FROM weather; 
2022-07-14 15:00:19,879 INFO sqlalchemy.engine.base.Engine {}


(1729957, 5)

In [32]:
data_type('weather')

2022-07-14 15:09:56,005 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-14 15:09:56,006 INFO sqlalchemy.engine.base.Engine {'name': "\n    SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS \n    where table_schema = 'mk7zp' and table_name = 'weather';\n    "}
2022-07-14 15:09:56,009 INFO sqlalchemy.engine.base.Engine 
    SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS 
    where table_schema = 'mk7zp' and table_name = 'weather';
    
2022-07-14 15:09:56,010 INFO sqlalchemy.engine.base.Engine {}


,data_type
0,timestamp without time zone
1,bigint
2,bigint
3,bigint
4,text


### Problem 3 - Data Analysis

In [ ]:
#

In [49]:
query ='''
        SELECT  coalesce(EXTRACT(year FROM date) as year, station, 
                round(avg(max_temp_c)/10,3) as avg_max_temp_cels, 
                round(avg(min_temp_c)/10,3) as avg_min_temp_cels, 
                round(sum(precip_mm)/10,3) as sum_precip_centm
        FROM weather
        GROUP BY EXTRACT(year FROM date), station
        ;
    '''
with engine.connect() as connection:
    df_summary = pd.read_sql(query, connection)

2022-07-14 15:26:35,459 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-14 15:26:35,460 INFO sqlalchemy.engine.base.Engine {'name': '\n        SELECT  EXTRACT(year FROM date) as year, station, \n                round(avg(max_temp_c)/10,3) as avg_max_temp_cels, \n                rou ... (52 characters truncated) ...              round(sum(precip_mm)/10,3) as sum_precip_centm\n        FROM weather\n        GROUP BY EXTRACT(year FROM date), station\n        ;\n    '}
2022-07-14 15:26:35,463 INFO sqlalchemy.engine.base.Engine 
        SELECT  EXTRACT(year FROM date) as year, station, 
                round(avg(max_temp_c)/10,3) as avg_max_temp_cels, 
                round(avg(min_temp_c)/10,3) as avg_min_temp_cels, 
                round(sum(precip_mm)/10,3) as sum_precip_centm
        FROM weather
        GROUP BY EXTRACT(year FROM date), station
        ;

In [50]:
df_summary.shape

(4820, 5)

In [51]:
df_summary = df_summary.sort_values('year').reset_index(drop=True)

In [52]:
df_summary.head(3)

,year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm
0,1985.0,USC00110072,15.335,-1.176,780.1
1,1985.0,USC00252205,14.917,2.718,705.1
2,1985.0,USC00252820,15.702,2.943,835.9


In [53]:
df_summary['year'] = df_summary['year'].astype('int64') 

In [54]:
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4820 entries, 0 to 4819
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               4820 non-null   int64  
 1   station            4820 non-null   object 
 2   avg_max_temp_cels  4820 non-null   float64
 3   avg_min_temp_cels  4820 non-null   float64
 4   sum_precip_centm   4820 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 188.4+ KB


#### Save the SQL result to 'weather_year_database' table in database

In [58]:
# drop_table('weather_year_summary')

In [57]:
create_table_stmt('weather_year_summary', df_summary, 'year','station' )  # db table name, dataframe, primary key 1, key 2

167
2022-07-14 15:28:07,739 INFO sqlalchemy.engine.base.Engine CREATE TABLE weather_year_summary(
	year BIGINT,
	station TEXT,
	avg_max_temp_cels REAL,
	avg_min_temp_cels REAL,
	sum_precip_centm REAL,
	PRIMARY KEY(year,station)
);
2022-07-14 15:28:07,740 INFO sqlalchemy.engine.base.Engine {}
2022-07-14 15:28:07,777 INFO sqlalchemy.engine.base.Engine COMMIT


In [60]:
ingest_data('weather_year_summary', df_summary)

Index(['year', 'station', 'avg_max_temp_cels', 'avg_min_temp_cels',
       'sum_precip_centm'],
      dtype='object')
	
year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm
%s,%s,%s,%s,%s
INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,275 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,276 INFO sqlalchemy.engine.base.Engine (1985, 'USC00110072', 15.335, -1.176, 780.1)
2022-07-14 15:28:34,279 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,281 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,282 INFO sqlalchemy.engine.base.Engine (1985, 'USC00252205', 14.917, 2.718, 705.1)
2022-07-14 15:28:34,283 INFO sqla

2022-07-14 15:28:34,367 INFO sqlalchemy.engine.base.Engine (1985, 'USC00134894', 14.941, 2.238, -261.7)
2022-07-14 15:28:34,368 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,370 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,370 INFO sqlalchemy.engine.base.Engine (1985, 'USC00135769', -3.811, -22.904, -5230.4)
2022-07-14 15:28:34,371 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,373 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,373 INFO sqlalchemy.engine.base.Engine (1985, 'USC00135796', 15.45, -1.316, 974.3)
2022-07-14 15:28:34,375 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,376 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_c

2022-07-14 15:28:34,457 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,459 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,459 INFO sqlalchemy.engine.base.Engine (1985, 'USC00335041', 16.951, 4.433, -4816.7)
2022-07-14 15:28:34,460 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,462 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,463 INFO sqlalchemy.engine.base.Engine (1985, 'USC00335315', 16.081, 3.505, 98.1)
2022-07-14 15:28:34,464 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,466 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,466 INFO sqlalchemy.eng

2022-07-14 15:28:34,546 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,546 INFO sqlalchemy.engine.base.Engine (1985, 'USC00257715', 15.825, 4.34, 706.0)
2022-07-14 15:28:34,548 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,549 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,550 INFO sqlalchemy.engine.base.Engine (1985, 'USC00258395', 13.559, 0.285, -293.4)
2022-07-14 15:28:34,551 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,553 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,554 INFO sqlalchemy.engine.base.Engine (1985, 'USC00258465', 16.184, 3.516, 790.2)
2022-07

2022-07-14 15:28:34,635 INFO sqlalchemy.engine.base.Engine (1985, 'USC00118147', 18.735, 7.594, 1373.4)
2022-07-14 15:28:34,637 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,640 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,640 INFO sqlalchemy.engine.base.Engine (1985, 'USC00118740', 15.827, 5.035, 1160.9)
2022-07-14 15:28:34,641 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,643 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:34,644 INFO sqlalchemy.engine.base.Engine (1985, 'USC00118916', 14.57, 2.302, 1061.5)
2022-07-14 15:28:34,645 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:34,647 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels

...

2022-07-14 15:28:51,155 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,157 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,158 INFO sqlalchemy.engine.base.Engine (2014, 'USC00138296', 12.585, 1.029, 1020.8)
2022-07-14 15:28:51,159 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,161 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,161 INFO sqlalchemy.engine.base.Engine (2014, 'USC00331072', 14.072, 2.435, 790.0)
2022-07-14 15:28:51,162 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,164 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,164 INFO sqlalchemy.eng

2022-07-14 15:28:51,236 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,236 INFO sqlalchemy.engine.base.Engine (2014, 'USC00255565', 16.744, -4.933, 638.2)
2022-07-14 15:28:51,237 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,239 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,239 INFO sqlalchemy.engine.base.Engine (2014, 'USC00256135', 14.165, 1.27, 641.0)
2022-07-14 15:28:51,240 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,242 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,242 INFO sqlalchemy.engine.base.Engine (2014, 'USC00256570', -56.225, -68.808, -28350.3)
2

2022-07-14 15:28:51,312 INFO sqlalchemy.engine.base.Engine (2014, 'USC00116558', 10.938, 0.082, -2828.7)
2022-07-14 15:28:51,313 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,315 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,315 INFO sqlalchemy.engine.base.Engine (2014, 'USC00116526', 0.178, -13.697, -7026.9)
2022-07-14 15:28:51,316 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,318 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,318 INFO sqlalchemy.engine.base.Engine (2014, 'USC00116446', 16.661, 4.899, 1170.6)
2022-07-14 15:28:51,319 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,321 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_

2022-07-14 15:28:51,397 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,399 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,400 INFO sqlalchemy.engine.base.Engine (2014, 'USC00112348', -101.651, -110.482, -12347.3)
2022-07-14 15:28:51,401 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,403 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,404 INFO sqlalchemy.engine.base.Engine (2014, 'USC00119241', -6.605, -20.479, -1885.4)
2022-07-14 15:28:51,405 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,407 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,407 INFO sql

2022-07-14 15:28:51,489 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,490 INFO sqlalchemy.engine.base.Engine (2014, 'USC00120784', 15.842, 4.855, 1248.8)
2022-07-14 15:28:51,491 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,493 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,493 INFO sqlalchemy.engine.base.Engine (2014, 'USC00121030', 5.64, 1.7, 1048.5)
2022-07-14 15:28:51,495 INFO sqlalchemy.engine.base.Engine COMMIT
2022-07-14 15:28:51,497 INFO sqlalchemy.engine.base.Engine INSERT INTO weather_year_summary (year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm) values (%s,%s,%s,%s,%s)
2022-07-14 15:28:51,497 INFO sqlalchemy.engine.base.Engine (2014, 'USC00121747', 13.891, -0.441, 172.8)
2022-07-

In [ ]:
# use sql query to get the table in the database.
display_db_table('weather_year_summary')

In [61]:
# use sql query to get the table in the database.
display_db_table('weather_year_summary')

2022-07-14 15:28:51,550 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-14 15:28:51,550 INFO sqlalchemy.engine.base.Engine {'name': ' SELECT * FROM weather_year_summary; '}
2022-07-14 15:28:51,553 INFO sqlalchemy.engine.base.Engine  SELECT * FROM weather_year_summary; 
2022-07-14 15:28:51,553 INFO sqlalchemy.engine.base.Engine {}


,year,station,avg_max_temp_cels,avg_min_temp_cels,sum_precip_centm
0,1985,USC00110072,15.335,-1.176,780.1
1,1985,USC00252205,14.917,2.718,705.1
2,1985,USC00252820,15.702,2.943,835.9
3,1985,USC00252840,15.901,3.825,-371.4
4,1985,USC00253035,15.967,2.343,-3378.8
...,...,...,...,...,...
4815,2014,USC00126001,-43.557,-36.578,-14999.9
4816,2014,USC00126580,13.538,1.618,278.9
4817,2014,USC00126705,8.732,-8.623,-8815.1
4818,2014,USC00124181,14.239,2.511,1055.9
